In [150]:
import os
import csv
import numpy as np
import pandas as pd
import zipfile
import re
import json

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [151]:
#unzip and read turns files into pandas df
zf1 = zipfile.ZipFile('./turns_part1.zip') 
turns1 = pd.read_csv(zf1.open('turns_part1.csv'))
zf2 = zipfile.ZipFile('./turns_part2.zip') 
turns2 = pd.read_csv(zf2.open('turns_part2.csv'))
zf3 = zipfile.ZipFile('./turns_part3.zip') 
turns3 = pd.read_csv(zf3.open('turns_part3.csv'))
zf4 = zipfile.ZipFile('./turns_part4.zip') 
turns4 = pd.read_csv(zf4.open('turns_part4.csv'))

#read summaries file into pandas df
summaries = pd.read_csv('summaries.csv')

In [152]:
# #use original scdb winning party as verdict
# #verdict value 0 = no favorable disposition for petitioning part apparent
# #verdict value 1 = petitioning party received a favorable disposition
# verdict = []
# verdict_csv = csv.reader(open('../SCDB_2017_01_caseCentered_Citation.csv'))
# for row in verdict_csv:
#     docket_number = re.sub('-', '_', row[13])
#     docket_number = re.sub(' ORIG', '_orig', docket_number)
# #     winning_party
#     case_id = row[10]+'_'+docket_number
#     verdict.append([case_id, row[36]])
    
# verdict_header = verdict.pop(0)
# verdict = pd.DataFrame(verdict, columns = verdict_header)
    
    
#concate the turns files
turns_combined = pd.concat([turns1, turns2, turns3, turns4])

#remove _t01 and _t02 from transcript_id in turns_combined
turns_combined['transcript_id'] = turns_combined['transcript_id'].str.replace('_t01','')
turns_combined['transcript_id'] = turns_combined['transcript_id'].str.replace('_t02','')

# #pivot turns files by transcript_id and speaker_role for texts
# speaker_role_text_pivot = turns_combined.pivot_table(index = 'transcript_id', 
#                                                      columns = 'speaker_role', 
#                                                      values = 'text',
#                                                      aggfunc=lambda x: ' '.join(x))

# #reset index
# speaker_role_text_pivot = speaker_role_text_pivot.reset_index()

# #join verdict into df
# train_test_df = speaker_role_text_pivot.join(verdict.set_index('term_docket')['partyWinning'], on='transcript_id')

# #remove NAs and blanks (these give errors when vectorizing)
# train_test_df = train_test_df.dropna()

# #create train and test split
# x = train_test_df[['not_a_justice', 'scotus_justice']]
# y = train_test_df.partyWinning
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)

# #count vectorizer
# count_vect = CountVectorizer()
# x_train_cv_not_a_justice = count_vect.fit(x_train['not_a_justice'])
# x_train_cv_scotus_justice = count_vect.fit(x_train['scotus_justice'])
# x_test_cv_not_a_justice = count_vect.fit(x_test['not_a_justice'])
# x_test_cv_scotus_justice = count_vect.fit(x_test['scotus_justice'])

# x_train_cv_not_a_justice = pd.DataFrame(x_train_cv_not_a_justice.transform(x_train['not_a_justice']).todense(),
#                                         columns = x_train_cv_not_a_justice.get_feature_names())
# x_train_cv_scotus_justice = pd.DataFrame(x_train_cv_scotus_justice.transform(x_train['scotus_justice']).todense(),
#                                         columns = x_train_cv_scotus_justice.get_feature_names())
# x_test_cv_not_a_justice = pd.DataFrame(x_test_cv_not_a_justice.transform(x_test['not_a_justice']).todense(),
#                                         columns = x_test_cv_not_a_justice.get_feature_names())
# x_test_cv_scotus_justice = pd.DataFrame(x_test_cv_scotus_justice.transform(x_test['scotus_justice']).todense(),
#                                         columns = x_test_cv_scotus_justice.get_feature_names())

# #concatenate the not_a_justice bow and scotus_justice bow
# x_train = pd.concat([x_train_cv_not_a_justice, x_train_cv_scotus_justice], axis = 1)
# x_test = pd.concat([x_test_cv_not_a_justice, x_test_cv_scotus_justice], axis = 1)

# #random forest
# forest = RandomForestClassifier(n_estimators = 100)
# forest = forest.fit(x_train, y_train)
# random_forest_prediction = forest.predict(x_test)
# rf_accuracy = np.mean(random_forest_prediction == y_test)
# print 'Accuracy of Bag-of-Words Random Forest Model:', round(rf_accuracy,4)

# # Multinomial Naive Baynes
# nb = MultinomialNB(alpha = 0.1).fit(x_train, y_train)
# naive_baynes_prediction = nb.predict(x_test)
# nb_accuracy = np.mean(naive_baynes_prediction == y_test)
# print 'Accuracy of Bag-of-Words Multinomial Naive Baynes Model:', round(nb_accuracy,4)

# # logistic regression
# lr_model = LogisticRegression(C = 1, penalty = "l2")
# lr_model_train = lr_model.fit(x_train, y_train)
# lr_prediction = lr_model.predict(x_test)
# lr_accuracy = np.mean(lr_prediction == y_test)
# print 'Accuracy of Bag-of-Words Logistic Regression Model:', round(lr_accuracy,4)

# print 'Actual Count of y_test Verdicts:\n', y_test.value_counts()

# print 'LR predictions:\n', np.unique(lr_prediction, return_counts=True)

In [153]:
# Extracting which side the advocates are representing from summaries dataframe

# Keeping track of things
all_records = 0
need_more = 0
total_cases = 0
cases_w_adv = 0
missing_adv = {}
advocate_dict = {}

# Go through each case and extract advocate list 
for row in summaries.values:
    total_cases += 1
    advocate_list = row[10]

    # ignore cases where the advocate list in nan
    if not isinstance(advocate_list, float):
        
        # extracting needed information from each advocate list
        first_party = row[6]
        second_party = row[8]
        transcript_id = re.sub("OA", "", row[1])
        print row[0], transcript_id
        
        # count cases where there is an advocate list
        cases_w_adv += 1
        
        # some test handling with regular expressions to extract names cleanly
        advocate_list =  re.sub('\[', '', advocate_list)
        advocate_list =  re.sub(' *\(', '', advocate_list)
        advocate_list =  re.sub('\)]', '', advocate_list).split("),")   
#         print advocate_list

        # Dictionaries to keep track of advocates 
        local_adv_dict = {}
        missing_adv[transcript_id] = []
        
        # Iterate through each advocate in an advocate list
        for advocate in advocate_list:
            
#             print advocate
            all_records += 1
            advocate_name = advocate.split(", ")[0]
        
            # If statements to assign sides to each advocate based on contents of summary file
            if re.search("appellant|Appellant", advocate) or re.search("petitioner|Petitioner", advocate):
                advocate_for = "appellant/petitioner"
            elif re.search("appellee|Appellee", advocate) or re.search("respondent|Respondent", advocate):
                advocate_for = "appellee/respondent"
            
            elif not isinstance(first_party, float) and re.search(first_party, advocate):
                advocate_for = "appellant/petitioner"          
            elif not isinstance(second_party, float) and re.search(second_party, advocate):
                advocate_for = "appellee/respondent"        
            else:
                advocate_for = "NEED MORE INFO"
                need_more += 1
                missing_adv[transcript_id].append([advocate_name, advocate])
#                 print advocate
                
#                 print "Advocate:", advocate, "\nFirst Party:", first_party, "Second Party:", second_party, "\n"

            # populating advocate_dict to fill out turns dataframe and missing_adv dict for cases needing more work
            local_adv_dict[advocate_name] = advocate_for
            advocate_dict[transcript_id] = local_adv_dict
           
        
print "still need to figure out", need_more, "records out of", all_records, "covering", cases_w_adv, "cases where the advocates are known"
# print "Number of cases with missing advocate label", len(missing_adv)

2 1970_79
3 1968_2
4 1981_81_411
5 1991_90_1262
8 1972_71_895
9 1995_94_1387
10 2009_08_1555
11 1958_3
14 1973_72_734
15 1956_257
17 2006_04_1350
18 1977_77_452
19 1995_94_1239
20 1985_84_1479
21 1981_80_965
22 1998_98_404
23 1995_95_345
24 1997_96_7901
25 1992_91_7328
26 1982_81_1857
27 1995_95_232
28 1976_75_6521
29 1986_85_889
30 2010_09_1163
31 1973_73_439
35 1979_79_677
37 1985_85_632
39 1977_76_1836
40 1975_74_100
41 1989_89_1433
42 2008_07_773
43 1981_78_1545
44 2007_06_9130
45 2006_06_102
49 2006_05_1342
50 1999_98_942
51 1989_88_805
52 1975_74_773
53 1994_94_514
54 1975_75_44
55 1991_90_6704
57 1989_89_530
58 1974_73_1942
60 2002_02_695
62 1984_83_2161
64 1976_75_1721
66 1973_73_679
67 1965_5
68 2010_09_11556
70 1980_79_1013
71 1990_89_1905
72 1989_88_1972
74 1977_76_5935
75 1977_76_938
76 1975_74_850
77 1978_78_5414
78 1984_84_325
79 1991_91_5843
80 1973_72_1557
81 1979_79_198
82 1961_242
83 1971_71_5313
84 1980_80_120
87 1974_73_1309
88 1973_72_948
89 1994_94_455
90 1978_78_

In [141]:
# summaries[213:216]
# turns_combined

# C = 0
# for l in missing_adv.values():
#     if len(l) == 2:
#         C +=1
# print C

# advocate_dict

In [11]:
# groupby1 = pd.DataFrame(turns_combined[turns_combined['speaker_role'] == 'not_a_justice'].groupby(['transcript_id', 'speaker'])['text_start'].min())
# groupby1.groupby(['transcript_id')['text_start'].min()
# # duration_df = pd.DataFrame(turns_df.groupby(['transcript_id', 'speaker_ID','speaker','speaker_role'])['text_duration'].sum())
# turns_combined['advocate_side'] = None

In [175]:
count = 0
for case in missing_adv:
    if len(advocate_dict[case]) == 2 and len(missing_adv[case]) > 0:
        count += 1
        print count, case, "\nFrom summaries:"
        for i in summaries[summaries['transcript_id'] == 'OA' + case]['advocates']:
            print "\t\t", i
        
        print "From missing_adv:"
        for adv in missing_adv[case]:
            print "\t\t", adv
        print "From advocate_dict:\n\t\t", advocate_dict[case], "\n"





1 1976_75_503 
From summaries:
		[(George Colvin Cochran, ), (Will A. Hickman, )]
From missing_adv:
		['George Colvin Cochran', 'George Colvin Cochran, ']
		['Will A. Hickman', 'Will A. Hickman, ']
From advocate_dict:
		{'Will A. Hickman': 'NEED MORE INFO', 'George Colvin Cochran': 'NEED MORE INFO'} 

2 1977_76_1171 
From summaries:
		[(William R. Quinlan, ), (Robert Masur, )]
From missing_adv:
		['William R. Quinlan', 'William R. Quinlan, ']
		['Robert Masur', 'Robert Masur, ']
From advocate_dict:
		{'Robert Masur': 'NEED MORE INFO', 'William R. Quinlan': 'NEED MORE INFO'} 

3 1975_74_1267 
From summaries:
		[(Miriam Naveira De Rodon, ), (Max Ramirez De Arellano, )]
From missing_adv:
		['Miriam Naveira De Rodon', 'Miriam Naveira De Rodon, ']
		['Max Ramirez De Arellano', 'Max Ramirez De Arellano, ']
From advocate_dict:
		{'Max Ramirez De Arellano': 'NEED MORE INFO', 'Miriam Naveira De Rodon': 'NEED MORE INFO'} 

4 1978_78_752 
From summaries:
		[(A. W. Sorelle III, ), (Douglas R. Lars